<a href="https://colab.research.google.com/github/ibrezm1/MultilayerKeras/blob/master/Text_Clssification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://www.opencodez.com/python/text-classification-using-keras.htm
#https://github.com/pavansolapure/opencodez-samples/blob/master/keras-text-classification/20_news_group_classification.py
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

In [0]:
from sklearn.datasets import fetch_20newsgroups
files_train = fetch_20newsgroups(subset='train')

In [79]:
files_train.target

array([7, 4, 4, ..., 3, 1, 8])

In [80]:
files_train.data[:5]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [82]:
label_names[5:]

['comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [0]:
data = pd.DataFrame(files_train.data, columns=['news'])
#df['target'] = pd.Series(cancer.target)

#data['tags'] = pd.Series(files_train.target_names)
data['category'] = pd.Series(files_train.target)
data['filename'] = pd.Series(files_train.filenames)

In [84]:
data.dropna(inplace=True)
data.shape

(11314, 3)

In [45]:
files_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [49]:
files_train.target_names[1]

'comp.graphics'

In [0]:
 data['category']= data['category'].apply(lambda x: files_train.target_names[x])

In [98]:
data.head()

,news,category,filename
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,rec.autos,/root/scikit_learn_data/20news_home/20news-byd...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,comp.sys.mac.hardware,/root/scikit_learn_data/20news_home/20news-byd...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,comp.sys.mac.hardware,/root/scikit_learn_data/20news_home/20news-byd...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,comp.graphics,/root/scikit_learn_data/20news_home/20news-byd...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,sci.space,/root/scikit_learn_data/20news_home/20news-byd...


In [97]:
print(data['category'][0])
print(data['news'][0])

rec.autos
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [101]:
# 20 news groups
num_labels = 20
vocab_size = 1500
batch_size = 100


train_size = int(len(data) * .8)

train_posts = data['news'][:train_size]
train_tags = data['category'][:train_size]
train_files_names = data['filename'][:train_size]

test_posts = data['news'][train_size:]
test_tags = data['category'][train_size:]
test_files_names = data['filename'][train_size:]

# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,)
tokenizer.fit_on_texts(train_posts)

x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=2,
                    verbose=1,
                    validation_split=0.1)

# creates a HDF5 file 'my_model.h5'
model.model.save('my_model.h5')

# Save Tokenizer i.e. Vocabulary
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)

print('Test accuracy:', score[1])

text_labels = encoder.classes_

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction[0])]
    print(test_files_names.iloc[i])
    print(test_posts.iloc[i])
    print('Actual label:' , test_tags.iloc[i])
    print("Predicted label: " , predicted_label)
    print('---------------------------------')

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 512)               768512    
_________________________________________________________________
activation_34 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_71 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_35 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_72 (Dense)             (None, 20)              

/usr/local/lib/python3.6/dist-packages/keras/engine/sequential.py:111: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


2263/2263 [==============================] - 0s 52us/step
Test accuracy: 0.7874502917332719
/root/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.hockey/53768
From: dchhabra@stpl.ists.ca (Deepak Chhabra)
Subject: Re: div. and conf. names
Nntp-Posting-Host: stpl.ists.ca
Organization: Solar Terresterial Physics Laboratory, ISTS
Distribution: na
Lines: 56

In article <1993Apr19.003221.11964@ramsey.cs.laurentian.ca> maynard@ramsey.cs.laurentian.ca (Roger Maynard) writes:
[Evan Pritchard writes:]

>>	I think that you are incorrect, Roger.  Patrick,
>>Smythe and Adams all played or coached in the league before becoming
>>front office types.  Hence, they did help build the league, although
>>they were not great players themselves.  
>
>Punch Imlach's contributions as a coach and GM were far greater than
>those of the above combined.  Should we name a division or trophy after
>him?  [owner vs. player contributions deleted...]
>Are we going to honour those who contributed to the lea

In [65]:
text_labels

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')

In [102]:
import keras
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense

inputA = Input(shape=(vocab_size,))
inputB = Input(shape=(2,))
# the first branch operates on the first input
x = Dense(8, activation="relu")(inputA)
x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = Dense(2, activation="relu")(inputB)
y = Dense(1, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
#z = Dense(2, activation="relu")(combined)
#z = Dense(1, activation="linear")(z)

z = Dense(num_labels, activation="softmax")(combined)

#model.add(Dense(num_labels))
#model.add(Activation('softmax'))

# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 1500)         0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
dense_73 (Dense)                (None, 8)            12008       input_15[0][0]                   
__________________________________________________________________________________________________
dense_75 (Dense)                (None, 2)            6           input_16[0][0]                   
___________________________________________________________________________________________